In [6]:
# make dataframes from txt data
# each txt contains one tagesschau transcription
import os
import pandas as pd
import numpy as np
from wordcloud import WordCloud
from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation as LDA
import warnings
import time
warnings.simplefilter("ignore", DeprecationWarning)
import matplotlib.pyplot as plt
import spacy
import de_core_news_lg
nlp = de_core_news_lg.load(disable=['parser'])
#nlp.add_pipe(nlp.create_pipe('sentencizer'))
import datetime
from joblib import Parallel, delayed
from functools import partial
from gensim import corpora,models
import re
import pyLDAvis.gensim
from gensim.utils import simple_preprocess
from gensim.test.utils import common_corpus
from gensim.models import LdaSeqModel
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
# Enable in-notebook visualizations
%matplotlib inline
pyLDAvis.enable_notebook()
pd.options.display.max_rows = 10


# load the s2t transcriptions
def load_transcriptions(path):
    os.chdir(path)
    transcriptions = os.listdir()#[:30]
    return transcriptions

# preprocessing for topic modelling, in this case LDA (latent dirichlet allocation)
def lemmatize_pipe(doc):
    lemma_list = [tok.lemma_ for tok in doc]# if str(tok).isalpha] #usually str(tok.lemma_).lower()
    return lemma_list

def preprocess_pipe(texts,nouns_array):#,non_nouns_array):
    preproc_pipe = []
    #non_nouns = non_nouns_array
    nounss = nouns_array
    for doc in nlp.pipe(texts, batch_size=int(len(transcriptions)/n_cpu_cores),n_process=int(n_cpu_cores)):
        preproc_pipe.append(lemmatize_pipe(doc))
        #non_nouns.extend([word for word in doc if word.pos_!='NOUN'])
        #nounss.append([str(chunk.text) for chunk in doc.noun_chunks])
        nounss.append([str(word) for word in doc if not word.is_stop and str(word).isalpha()])# and (word.pos_=='NOUN' or word.pos_=='PROPN')])# or word.pos_=='ADJ'])
    return preproc_pipe,nounss#,non_nouns

def preprocess_transcriptions(transcripts_df,number_cpu_cores=os.cpu_count()-1):
    
    manual_stopwords = ['wetter','wochentag','monat','ziel','zahl','stunden','weg','fernsehen','praesident',
                        'damen','herren','menschen','land','abend','grad','nacht','euro','geld','regierung',
                       'millionen','unternehmen','tagesthemen','angaben','himmelsrichtung',
                       'milliarden','tagesschau','woche','wochen','leben','%','welt','€','sueden',
                        'mitarbeiter','montag','welt','richtung','deutsche','deutschen','mal',
                        'moeglichkeit','fast','klar','meist','letzten','mehrere','steht','bleibt','sieht',
                        'deutlich','gestern','sehen','kritik','guten','folgen','neues','geben','bislang',
                       'eigentlich','teilweise','lassen','suedosten','gekommen','insgesamt','sagen','erneut',
                        'frauen','bringen','nachmittag','naechsten','mindestens','krise','freundlich','kuenftig',
                        'fall','videos','faellt','zukunft','gewalt','getoetet','melden','sicherheit','vater',
                        'gericht','bau','besucher','nebel','arbeit','familien','seiten','verhandlungen','genau',
                        'halten','liegen','hilfe','flammen']
    
    # init array for all nouns found by spacy
    nounss = []
    #print("Use: "+str(len(transcriptions))+" transcriptions, preprocess them (and perform LDA (Latent Dirichlet Allocation))")
    # use following line instead of the line after the folowing to use "tagesschau" only
    #df = pd.DataFrame(index=[i for i in range(len(transcriptions)) if "tagessschau" in transcriptions[i]], columns=['transcriptionName','content','year','month','day'])
    df = pd.DataFrame(index=[i for i in range(len(transcriptions))], columns=['transcriptionName','content','preprocessed','year','month','day'])
    for transcription in range(len(transcriptions)):
        # use following line to use "tagesschau" only
        #if "tagesschau" in str(transcriptions[transcription]):
        if transcription>-1: #placeholder for above line
            with open(transcriptions[transcription], "r") as f:
                df.loc[transcription,'transcriptionName'] = str(transcriptions[transcription])
                df.loc[transcription,'content'] = f.read()
    df = df.reset_index()
    df1=df.copy()
    for transcript in range(len(df)):
        # remove "newline" and punctuation
        #print(df.loc[transcript,'content'])
        df.loc[transcript,'content'] = df.loc[transcript,'content'].replace("\n","").replace(".","")

    stop_words = stopwords.words('german')
    for i in range(len(df)):
        df.at[i,'content'] = ([word for word in simple_preprocess((df.loc[i,'content'])) if word not in stop_words])
        df.at[i,'content'] = ','.join(df.loc[i,'content']).replace(","," ")
    #print(df.loc[0,'content'])
    
    #print(df['content'][0])

    sta = datetime.datetime.now()
    df['content'],nounss = preprocess_pipe(df['content'],nounss)#,non_nouns)
    
    # ab hier ist df['content'] unbedeutend -> nutze nounss
    
    #bigram
    listOfAllForBigram = []
    for i in range(len(nounss)):
        listOfAllForBigram.append(nounss[i])#.split(" "))
    #print(listOfAllForBigram[1])
    bigram = models.Phrases(listOfAllForBigram, min_count=20, threshold=50)
    trigram = models.Phrases(bigram[listOfAllForBigram], threshold=50)
    bigram_mod = models.phrases.Phraser(bigram)
    trigram_mod = models.phrases.Phraser(trigram)
    
    for i in range(len(nounss)):
        #if i == 0:
         #   print("before:",df.loc[i,'content'])
        nounss[i] = bigram_mod[nounss[i]]
        nounss[i] = trigram_mod[bigram_mod[nounss[i]]]
        '''if i == 0:
            for word in nounss[i]:
                if "_" in word:
                    print(word)
        '''
            #print("after:",df.loc[i,'content'])
        #nounss[i] = ','.join(nounss[i]).replace(","," ")

    '''    for i in range(len(nounss)):
        for j in nounss[i]:
            if "_" in j:
                print("ii",j)
    '''
    #for i in range(len(nounss)):
     #   nounss[i] = ','.join(nounss[i]).replace(","," ")
    
    
    #print(df['content'][0])
    print("lemmatizing+filtering out nouns needs:",(datetime.datetime.now()-sta).total_seconds(),"seconds")

    for i in range(len(nounss)):
        nounss[i] = [noun.lower() for noun in nounss[i]]

    manual = [[['wetter'],['sonne','regen','wind','schnee','schauer','luft','wolken','gewitter','gewittern']],
              [['himmelsrichtung'],['norden','sueden','osten','westen']],
              [['wochentag'],['montag','dienstag','mittwoch','donnerstag','freitag','samstag','sonntag']],
              [['monat'],['januar','februar','maerz','april','mai','juni','juli','august','september','oktober','november','dezember']]
             ,[['corona'],['korona']]]
    start_manual=datetime.datetime.now()
    for i in range(len(nounss)):
        for j in range(len(nounss[i])):
            nounss[i][j] = nounss[i][j].replace("ä",'ae').replace("ü","ue").replace("ö","oe").replace("ß","ss")
            for k in manual:
                for l in range(len(k[1])):
                    if k[1][l]==nounss[i][j]:
                        nounss[i][j] = k[0][0]
    end_manual=datetime.datetime.now()
    print("replacing e.g. 'sonne','regen'... by 'wetter' needs:",
          (end_manual-start_manual).total_seconds(),"seconds")    
    
    #"""
    manualStopWords = manual_stopwords
    removeWords = []
    for i in range(len(nounss)):
        for j in range(len(nounss[i])):
            for k in manualStopWords:
                if k in nounss[i][j]:
                    #print("nounss in manualS",nounss[i])
                    removeWords.append(nounss[i][j])
    removeWords = list(dict.fromkeys(removeWords))
    #"""
    for i in range(len(nounss)):
        nounss[i] = [word for word in nounss[i] if word not in removeWords]

    for i in range(len(nounss)):
        nounss[i] = (','.join(nounss[i])).replace(','," ")
    
    #merge all docs into dataframe
    for i in range(len(df)):
        df.at[i,'preprocessed'] = nounss[i]
        
    for i in range(len(df)):
        if 'tagesschau' in df.loc[i,'transcriptionName'] and 'vor' not in df.loc[i,'transcriptionName']:
            df.at[i,'year'] = int((df.loc[i,'transcriptionName'].split('_')[1][4:8]))
    #display(df.head(10))
    return df,df1

def print_workcloud(processed_transcripts):
    
    # used from tutorial to create a workcloud
    all_documents_as_one_string = ','.join(list(processed_transcripts.values))
    print(len(all_documents_as_one_string))
    #all_documents_as_one_string = ','.join(list(df['content'].values))
    wordcloud = WordCloud(width=800,height=400,background_color="black", max_words=500, contour_width=3, contour_color='steelblue')
    wordcloud.generate(all_documents_as_one_string)
    time.sleep(0.3)
    display(wordcloud.to_image())

def plot_25_most_common_words(count_data, count_vectorizer):
    words = count_vectorizer.get_feature_names()
    total_counts = np.zeros(len(words))
    for t in count_data:
        total_counts+=t.toarray()[0]
    
    count_dict = (zip(words, total_counts))
    count_dict = sorted(count_dict, key=lambda x:x[1], reverse=True)[0:25]
    #print((count_dict))
    words = [w[0] for w in count_dict]
    counts = [w[1] for w in count_dict]
    x_pos = np.arange(len(words)) 
    #print(x_pos)
    plt.figure(2, figsize=(15, 15/1.6180))
    plt.subplot(title='25 most common words')
    sns.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 2.5})
    dfPlot = pd.DataFrame(data={'words':words,'counts':counts})
    #display(dfPlot)
    sns.barplot(x=words,y=counts,data=dfPlot,palette='husl')
    plt.xticks(x_pos, words, rotation=90) 
    plt.xlabel('words')
    plt.ylabel('counts')
    plt.show()


def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % (int(topic_idx)+1)," ".join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    

def train_lda_sklearn(df,count_vectorizer,count_data,number_topics=10,number_words=3,ngram_tuple=(1,1),stop_words=None):#stopwords.words('german'))
    # n_jobs=-1 to use all processors, max_iter=50 randomly chosen, not sure about it
    lda = LDA(n_components=n_topics, n_jobs=-1)
    lda.fit(count_data)
    print("LDA Topics:")
    print_topics(lda, count_vectorizer, n_words)
    return lda

def train_model_gensim(df,num_topics=20,passes=10,workers=int((os.cpu_count()/2))-1,no_above=0.15):
    dfNew = df.copy()
    for i in range(len(df['preprocessed'])):
        dfNew.at[i,'preprocessed'] = df.loc[i,'preprocessed'].split(" ")
    dictionary = corpora.Dictionary(dfNew['preprocessed'])
    #print(dictionary)
    wordCorona = ([(dictionary[i],i) for i in range(len(dictionary)) if 'corona' in dictionary[i]])
    #print("corona vorher:",wordCorona)
    if len(wordCorona) > 0:
        print('The word "' + wordCorona[1][0] + '" appears in', dictionary.dfs[wordCorona[1][1]],'documents')
    word = ([(dictionary[i],i) for i in range(len(dictionary)) if 'eu' in dictionary[i]])
    #print(word)
    if len(word) > 0:
        print('The word "' + word[0][0] + '" appears in', dictionary.dfs[word[0][1]],'documents')
    print("number words in dictionary before:",len(dictionary))
    #dictionary.filter_n_most_frequent(10)
    #print("number words in dictionary after mostfreq:",len(dictionary))
    dictionary.filter_extremes(no_above=no_above)
    wordCorona = ([(dictionary[i],i) for i in range(len(dictionary)) if 'corona' in dictionary[i]])
    #print("corona nach no_above:",wordCorona)
    word = ([(dictionary[i],i) for i in range(len(dictionary)) if 'spd' in dictionary[i]])
    if len(word) > 0:
        print('The word "' + word[0][0] + '" appears in', dictionary.dfs[word[0][1]],'documents')
    print("number words in dictionary after noAbove:",len(dictionary))
    bow_corpus = [dictionary.doc2bow(doc) for doc in dfNew['preprocessed']]
    lda_model = models.LdaMulticore(bow_corpus, num_topics=num_topics, id2word=dictionary, passes=passes, workers=workers,minimum_probability=0.001)
    return lda_model, bow_corpus, dictionary

"""
def train_model_gensim_seq_lda(df,num_topics=20,passes=10,workers=int((os.cpu_count()/2))-10,yearsLen=[]):
    for i in range(len(df['preprocessed'])):
        df.at[i,'preprocessed'] = df.loc[i,'preprocessed'].split(" ")
    dictionary = corpora.Dictionary(df['preprocessed'])
    word = ([(dictionary[i],i) for i in range(len(dictionary)) if 'spd' in dictionary[i]])
    if len(word) > 0:
        print('The word "' + word[0][0] + '" appears in', dictionary.dfs[word[0][1]],'documents')
    print("number words in dictionary before:",len(dictionary))
    #dictionary.filter_n_most_frequent(10)
    #print("number words in dictionary after mostfreq:",len(dictionary))
    #dictionary.filter_extremes(no_above=0.2)
    word = ([(dictionary[i],i) for i in range(len(dictionary)) if 'spd' in dictionary[i]])
    if len(word) > 0:
        print('The word "' + word[0][0] + '" appears in', dictionary.dfs[word[0][1]],'documents')
    print("number words in dictionary after noAbove:",len(dictionary))
    bow_corpus = [dictionary.doc2bow(doc) for doc in df['preprocessed']]
    print("bow:",len(bow_corpus))
    print("common:",len(common_corpus))
    #lda_model = models.LdaMulticore(bow_corpus, num_topics=num_topics, id2word=dictionary, passes=passes, workers=workers)
    start_seq = datetime.datetime.now()
    ldaseq = LdaSeqModel(corpus=bow_corpus, id2word=dictionary, time_slice=yearsLen, num_topics=num_topics, chunksize=1)
    end_seq=datetime.datetime.now()
    print("lda_seq needs:",(end_seq - start_seq).total_seconds(),"seconds")
    return ldaseq, bow_corpus, dictionary
""" 

    

print("done")

done


In [7]:
#%%time
################### preprocess data for given daterange in yearly portions for feasibility in terms of RAM usage #######################
################### train on whole data afterwards #############################################################################################
"""
path = "/home/sim/all/Master/Forschungspraktikum/Tagesschau/transcripts/"
n_cpu_cores = 13 #int(os.cpu_count()-1)
print("number cpu_cores to use:",n_cpu_cores)
df_all_processed = pd.DataFrame(index=[], columns=['preprocessed'])


#load s2t transcriptions
path = "/home/sim/all/Master/Forschungspraktikum/Tagesschau/transcripts/"
transcriptions = load_transcriptions(path)
keys = ['2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020']
years_dict = {}
for year in keys: 
    years_dict[year] = []

# prepare daterange of interest for lda
daterange = pd.date_range(datetime.datetime(2007, 1, 1), datetime.datetime(2021, 1, 31))
dates_of_interest = []
for date in daterange:
    for i in transcriptions:
        if 'tagesschau' in i and 'vor' not in i:
            #print(i)
            year = int((i.split('_')[1][4:8]))
            if year==date.year:
                month = int((i.split('_')[1][2:4]))
                if month == date.month:
                    day = int((i.split('_')[1][0:2]))
                    if day == date.day:
                        years_dict[str(year)].append(i)
                        dates_of_interest.append(i)
print("There are ",len(dates_of_interest),"transcriptions in given daterange")
df_all_array = []
yearsLen = []
# preprocess every year
for year in years_dict.keys():
    print("\npreprocess transcriptions of year:",year,"quantity:",len(years_dict[str(year)]))
    if len(years_dict[str(year)]) > 0:
        yearsLen.append(len(years_dict[str(year)]))
    if len(years_dict[str(year)]) == 0:
        print("no transcription for year:",year)
        continue
    start = datetime.datetime.now()
    transcriptions = years_dict[str(year)]
    df_processed,df1 = preprocess_transcriptions(transcriptions,number_cpu_cores=n_cpu_cores)
    #print("MMMMMMMMMMM")
    #print(df_processed['preprocessed'])
    #print(len(df_processed['preprocessed']),len(df_all_processed['preprocessed']))
    for i in range(len(df_processed)):
        df_all_processed = df_all_processed.append({'preprocessed':df_processed.loc[i,'preprocessed']},ignore_index=True)
    #print(len(df_all_processed))
    #display(df_all_processed)
          
print("len of df:",len(df_all_processed))

print("yearsLen:",yearsLen)

'''
# Initialise count vectorizer
count_vectorizer = CountVectorizer(ngram_range=(1,1))

# Fit and transform the processed titles
count_data = count_vectorizer.fit_transform(df_all_processed['preprocessed'])

#params for LDA
n_topics = 10 #alpha
n_words = 5 #beta
'''
# Visualise the 25 most common words
#plot_25_most_common_words(count_data, count_vectorizer)

#"""
#no_above = [0.05,0.1,0.15,0.25,0.35,0.5,0.7,1]
no_above = [0.25,0.6,1]
for x in no_above:
    # with gensim lda
    print("Train LDA with no_above:",x,"...")
    startLDA = datetime.datetime.now()
    lda_model,bow_corpus,dictionary = train_model_gensim(df=df_all_processed,num_topics=20,passes=50,workers=7,no_above=x)
    #lda_model1,bow_corpus1,dictionary1 = train_model_gensim_seq_lda(df=df_all_processed,num_topics=20,passes=10,workers=7,yearsLen=yearsLen)
    endLDA = datetime.datetime.now()
    print("lda training needs:",(endLDA-startLDA).total_seconds(),"seconds")
    for idx,topic in lda_model.print_topics(-1):
        probs = ((topic.replace('"','').replace(" ","").split('+')))
        for prob in range(len(probs)):
            probs[prob] = (probs[prob].split("*"))
        probOver001 = [str(x[1])+'('+str(x[0])+')' for x in probs]# if float(x[0])>0.0005]
        print('Topic: {} \nWords: {}'.format(idx+1,probOver001))
        #print('Topic: {} \nWords: {}'.format(idx+1, re.sub('[^A-Za-z ]+', '', topic).replace("  "," ")))
    #"""
    """
    # with sklearn lda
    lda_model = train_lda_sklearn(df_processed,count_vectorizer,count_data,number_topics=n_topics,number_words=n_words,ngram_tuple=(2,3),stop_words=None)
    print_topics(lda_model, count_vectorizer, n_words)
    del df_processed
    del count_data
    del lda_model
    #print("loop needed:",(end-start).total_seconds(),"seconds")
    #"""
    #"""
    startVis = datetime.datetime.now()
    display(pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary))
    endVis = datetime.datetime.now()
    print((endVis-startVis).total_seconds(),"seconds")

print("\ndone")

Train LDA with no_above: 0.25 ...
The word "corona" appears in 280 documents
The word "betreuung" appears in 145 documents
number words in dictionary before: 149120
The word "spd_chef" appears in 123 documents
number words in dictionary after noAbove: 46525
lda training needs: 2475.868313 seconds
Topic: 1 
Words: ['opel(0.037)', 'general_motors(0.008)', 'magna(0.004)', 'iran(0.004)', 'bochum(0.004)', 'gm(0.004)', 'ruesselsheim(0.003)', 'werk(0.003)', 'fiat(0.003)', 'insolvenz(0.002)']
Topic: 2 
Words: ['militaer(0.002)', 'afghanistan(0.002)', 'hitler(0.002)', 'geloebnis(0.002)', 'stauffenberg(0.002)', 'ddr(0.001)', 'kopftuch(0.001)', 'bendlerblock(0.001)', 'vertreter(0.001)', 'obama(0.001)']
Topic: 3 
Words: ['ukraine(0.046)', 'kiew(0.016)', 'ukrainische(0.012)', 'krim(0.010)', 'ukrainischen(0.010)', 'separatisten(0.006)', 'russische(0.006)', 'moskau(0.006)', 'russischen(0.004)', 'janukowitsch(0.004)']
Topic: 4 
Words: ['nato(0.003)', 'banken(0.002)', 'bundeswehr(0.002)', 'buback(0.002

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
41    -0.044588 -0.035587       1        1  31.712475
11    -0.067983 -0.014397       2        1  14.768086
44    -0.049335  0.012548       3        1   7.199290
33    -0.038812  0.140957       4        1   6.219122
38    -0.050704  0.033301       5        1   5.407171
...         ...       ...     ...      ...        ...
4      0.105891 -0.025509      46        1   0.150229
28     0.147180 -0.006351      47        1   0.131774
37     0.241341  0.022103      48        1   0.105132
14     0.156204 -0.064805      49        1   0.104500
16     0.155526 -0.102706      50        1   0.078826

[50 rows x 5 columns], topic_info=              Term         Freq        Total Category  logprob  loglift
169         israel  2382.000000  2382.000000  Default  30.0000  30.0000
1221       ukraine  2152.000000  2152.000000  Default  29.0000  29.0000
44748       corona  1476.000000  1476.000000  Default  28.0000  28.0000
678    oesterreich   885.000000   885.000000  Default  27.0000  27.0000
21788          afd  1656.000000  1656.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
703        richter     3.532211  1438.870483  Topic50  -6.3927   1.1360
213         london     3.221109  1173.261597  Topic50  -6.4849   1.2479
2578        regeln     3.183539  1162.031128  Topic50  -6.4967   1.2458
1890        plaene     3.157109  1138.489624  Topic50  -6.5050   1.2579
302        steigen     3.039839  1009.321716  Topic50  -6.5429   1.3405

[3400 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
22062     32  0.789925        abba
0         10  0.963102       abbas
0         26  0.005963       abbas
0         32  0.017890       abbas
0         41  0.002982       abbas
...      ...       ...         ...
42529     15  0.924770   zyprische
40270      2  0.132819  zyprischen
40270     17  0.066409  zyprischen
40270     22  0.531274  zyprischen
40270     33  0.199228  zyprischen

[27396 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[42, 12, 45, 34, 39, 9, 24, 14, 32, 23, 21, 16, 28, 41, 46, 3, 33, 4, 6, 1, 27, 47, 7, 8, 49, 18, 37, 43, 22, 31, 44, 2, 36, 30, 11, 13, 20, 25, 40, 19, 48, 35, 26, 10, 50, 5, 29, 38, 15, 17])

42.001822 seconds
Train LDA with no_above: 0.6 ...
The word "corona" appears in 280 documents
The word "betreuung" appears in 145 documents
number words in dictionary before: 149120
The word "spd" appears in 2498 documents
number words in dictionary after noAbove: 46702
lda training needs: 2929.459444 seconds
Topic: 1 
Words: ['eu(0.005)', 'usa(0.003)', 'us(0.002)', 'treffen(0.002)', 'laender(0.002)', 'tuerkei(0.002)', 'parlament(0.001)', 'frage(0.001)', 'opposition(0.001)', 'syrien(0.001)']
Topic: 2 
Words: ['kosovo(0.043)', 'nsa(0.026)', 'serbien(0.024)', 'serbische(0.013)', 'serben(0.013)', 'serbischen(0.011)', 'pristina(0.005)', 'abgehoert(0.005)', 'geheimdienst(0.005)', 'unabhaengigkeit(0.004)']
Topic: 3 
Words: ['wulff(0.051)', 'christian_wulff(0.047)', 'wulf(0.014)', 'tuev(0.011)', 'gronewold(0.009)', 'qimonda(0.008)', 'pip(0.008)', 'vorteilsannahme(0.007)', 'implantate(0.007)', 'brustimplantate(0.006)']
Topic: 4 
Words: ['wasser(0.037)', 'hochwasser(0.024)', 'fluten(0.012)', 'w

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.057665 -0.063979       1        1  37.514187
16    -0.120748 -0.080321       2        1  11.541061
28    -0.116259 -0.064619       3        1  11.018950
34    -0.143041  0.050115       4        1   8.389429
37    -0.183864  0.046896       5        1   5.261751
...         ...       ...     ...      ...        ...
39     0.175941 -0.023399      46        1   0.023731
35     0.188393 -0.030016      47        1   0.020203
36     0.245191 -0.022249      48        1   0.004431
21     0.243164 -0.023353      49        1   0.003588
4      0.241807 -0.024016      50        1   0.003082

[50 rows x 5 columns], topic_info=              Term        Freq        Total Category  logprob  loglift
116             eu  9063.00000  9063.000000  Default  30.0000  30.0000
821            spd  7707.00000  7707.000000  Default  29.0000  29.0000
4803  fluechtlinge  3560.00000  3560.000000  Default  28.0000  28.0000
532            cdu  3854.00000  3854.000000  Default  27.0000  27.0000
1724           usa  6114.00000  6114.000000  Default  26.0000  26.0000
...            ...         ...          ...      ...      ...      ...
2034       polizei     0.00204  5049.354004  Topic50 -10.7515  -4.3344
193       jaehrige     0.00204  2460.875732  Topic50 -10.7515  -3.6157
1778         zuvor     0.00204  2431.379639  Topic50 -10.7515  -3.6036
3662         seite     0.00204  2239.528564  Topic50 -10.7515  -3.5214
665     inzwischen     0.00204  2725.762695  Topic50 -10.7515  -3.7179

[3276 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
0          1  0.022793      abbas
0          2  0.019944      abbas
0          3  0.128209      abbas
0          4  0.424514      abbas
0         12  0.008547      abbas
...      ...       ...        ...
15190     14  0.007502     zypern
15190     22  0.041262     zypern
15190     33  0.003751     zypern
15190     35  0.003751     zypern
42706      2  0.961267  zyprische

[19486 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 17, 29, 35, 38, 31, 19, 18, 46, 23, 8, 14, 21, 25, 49, 13, 26, 39, 30, 45, 42, 33, 9, 27, 4, 2, 20, 16, 34, 41, 12, 24, 28, 43, 47, 6, 48, 10, 3, 50, 11, 44, 7, 15, 32, 40, 36, 37, 22, 5])

46.887985 seconds
Train LDA with no_above: 1 ...
The word "corona" appears in 280 documents
The word "betreuung" appears in 145 documents
number words in dictionary before: 149120
The word "spd" appears in 2498 documents
number words in dictionary after noAbove: 46703
lda training needs: 2666.48267 seconds
Topic: 1 
Words: ['spd(0.026)', 'cdu(0.018)', 'partei(0.017)', 'fdp(0.014)', 'gruenen(0.013)', 'koalition(0.010)', 'wahl(0.010)', 'parteien(0.008)', 'afd(0.007)', 'union(0.006)']
Topic: 2 
Words: ['ebooks(0.002)', 'einzelspieler(0.001)', 'berlin(0.000)', 'eu(0.000)', 'partei(0.000)', 'spd(0.000)', 'polizei(0.000)', 'us(0.000)', 'usa(0.000)', 'lage(0.000)']
Topic: 3 
Words: ['klimaschutz(0.007)', 'co(0.004)', 'gipfel(0.004)', 'usa(0.004)', 'eu(0.003)', 'klima(0.003)', 'china(0.003)', 'kosten(0.003)', 'spd(0.003)', 'koalition(0.002)']
Topic: 4 
Words: ['ceta(0.010)', 'eu(0.005)', 'spd(0.004)', 'tafeln(0.003)', 'bamf(0.003)', 'berlin(0.003)', 'eu_kanada(0.003)', 'usa(0.002)', 'irak(0.00

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
23    -0.051265 -0.014977       1        1  28.426456
25    -0.069968 -0.083305       2        1  27.970833
47    -0.112149 -0.015005       3        1   7.733801
0     -0.144509 -0.048936       4        1   4.730780
44    -0.133762 -0.068384       5        1   4.275949
...         ...       ...     ...      ...        ...
20     0.207888 -0.020714      46        1   0.027624
43     0.211062 -0.015613      47        1   0.026657
45     0.258326 -0.023223      48        1   0.003321
1      0.257895 -0.023454      49        1   0.003169
48     0.257610 -0.023582      50        1   0.003075

[50 rows x 5 columns], topic_info=           Term        Freq        Total Category  logprob  loglift
117          eu  9089.00000  9089.000000  Default  30.0000  30.0000
822         spd  7730.00000  7730.000000  Default  29.0000  29.0000
864     tuerkei  3706.00000  3706.000000  Default  28.0000  28.0000
2035    polizei  5049.00000  5049.000000  Default  27.0000  27.0000
1332    ukraine  2097.00000  2097.000000  Default  26.0000  26.0000
...         ...         ...          ...      ...      ...      ...
2176      zeigt     0.00204  2592.592529  Topic50 -10.7515  -3.6657
349      stehen     0.00204  3402.389160  Topic50 -10.7515  -3.9375
194    jaehrige     0.00204  2462.811768  Topic50 -10.7515  -3.6144
3134     zeigen     0.00204  2519.082764  Topic50 -10.7515  -3.6369
978   behoerden     0.00204  2697.127441  Topic50 -10.7515  -3.7052

[3331 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
0          1  0.266128           abbas
0          2  0.014622           abbas
0          4  0.038018           abbas
0          5  0.005849           abbas
0          8  0.002924           abbas
...      ...       ...             ...
22017     32  0.015030   zwischenlager
22017     34  0.210423   zwischenlager
16347      2  0.578717  zwoelfjaehrige
16347     32  0.144679  zwoelfjaehrige
16347     35  0.289358  zwoelfjaehrige

[20430 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[24, 26, 48, 1, 45, 3, 7, 15, 9, 31, 17, 6, 50, 12, 22, 32, 23, 18, 40, 34, 11, 14, 19, 35, 10, 29, 25, 4, 20, 43, 41, 5, 13, 37, 16, 38, 28, 39, 33, 47, 8, 27, 36, 42, 30, 21, 44, 46, 2, 49])

43.958335 seconds

done


In [ ]:
#,'france','','','','','','','',''
'''
from gensim.models.wrappers.dtmmodel import DtmModel

doc_topic, topic_term, doc_lengths, term_frequency, vocab = lda_model1.dtm_vis(time=0, corpus=bow_corpus1)
vis_dtm = pyLDAvis.prepare(topic_term_dists=topic_term, doc_topic_dists=doc_topic, doc_lengths=doc_lengths, vocab=vocab, term_frequency=term_frequency)
pyLDAvis.display(vis_dtm)
lda_model1.print_topics(time=0)'''
doc = nlp("Hamburg ist eine schöne Stadt, Dresden auch. Die Hamas in France ist stark.")

for token in doc:
    1==1
    #print(token.text, token.pos_, token.tag_, token.dep_,
             token.is_stop)

In [ ]:
#%%time
############################### preprocess and train for each year separately for a given daterange ####################################

'''
path = "/home/sim/all/Master/Forschungspraktikum/Tagesschau/transcripts/"
n_cpu_cores = 13 #int(os.cpu_count()-1)
print("number cpu_cores to use:",n_cpu_cores)

#load s2t transcriptions
path = "/home/sim/all/Master/Forschungspraktikum/Tagesschau/transcripts/"
transcriptions = load_transcriptions(path)
keys = ['2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020']
years_dict = {}
for year in keys: 
    years_dict[year] = []

# lda for certain dates
daterange = pd.date_range(datetime.datetime(2007, 1, 1), datetime.datetime(2020, 7, 31))
dates_of_interest = []
for date in daterange:
    for i in transcriptions:
        if 'tagesschau' in i and 'vor' not in i:
            year = int((i.split('_')[1][4:8]))
            if year==date.year:
                month = int((i.split('_')[1][2:4]))
                if month == date.month:
                    day = int((i.split('_')[1][0:2]))
                    if day == date.day:
                        years_dict[str(year)].append(i)
                        dates_of_interest.append(i)
print("There are ",len(dates_of_interest),"transcriptions in given daterange")

#"""
for year in years_dict.keys():
    print("\npreprocess transcriptions of year:",year,"quantity:",len(years_dict[str(year)]))
    if len(years_dict[str(year)]) == 0:
        print("no transcription for year:",year)
        continue
    start = datetime.datetime.now()
    #transcriptions = transcriptions[i*50:(i+1)*50]
    transcriptions = years_dict[str(year)]
    # maybe: more threads -> more RAM so in case of low ram use less cores


    df_processed,df1 = preprocess_transcriptions(transcriptions,number_cpu_cores=n_cpu_cores)
    #print_workcloud(df_processed['preprocessed'])

    # Initialise count vectorizer
    count_vectorizer = CountVectorizer(ngram_range=(1,1))
    # Fit and transform the processed titlest
    count_data = count_vectorizer.fit_transform(df_processed['preprocessed'])
    #params for LDA
    n_topics = 10 #alpha
    n_words = 5 #beta

    # Visualise the 10 most common words
    #plot_25_most_common_words(count_data, count_vectorizer)
    
    #"""
    # with gensim lda
    startLDA = datetime.datetime.now()
    lda_model,bow_corpus,dictionary = train_model_gensim(df=df_processed,num_topics=20,passes=10,workers=7)
    endLDA = datetime.datetime.now()
    print("lda training needs:",(endLDA-startLDA).total_seconds(),"seconds")
    for idx, topic in lda_model.print_topics(-1):
        print('Topic: {} \nWords: {}'.format(idx+1, re.sub('[^A-Za-z ]+', '', topic).replace("  "," ")))
    
    #"""
    """
    # with sklearn lda
    lda_model = train_lda_sklearn(df_processed,count_vectorizer,count_data,number_topics=n_topics,number_words=n_words,ngram_tuple=(2,3),stop_words=None)
    print_topics(lda_model, count_vectorizer, n_words)
    del df_processed
    del count_data
    del lda_model
    #print("loop needed:",(end-start).total_seconds(),"seconds")
    #"""
    startVis = datetime.datetime.now()
    display(pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary))
    endVis = datetime.datetime.now()
    print((endVis-startVis).total_seconds(),"seconds")
'''
#"""

In [ ]:
print(common_corpus)
#%%time 
#lda_model = models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=100, workers=7)
#for idx, topic in lda_model.print_topics(-1):
#    print('Topic: {} \nWords: {}'.format(idx+1, re.sub('[^A-Za-z ]+', '', topic).replace("  "," ")))

In [ ]:
#%%time 
#display(pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary))